In [47]:
import csv
from datetime import datetime, tzinfo, timedelta
from dateutil import tz
import calendar
import time

In [48]:
def read_csv_data(csvfile):
    all_data = []
    with open(csvfile, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            all_data.append(row)
    return all_data

In [49]:
 def get_sec(time_str):
    time_str = str(time_str)
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

#we get the submit times, and get the time the Empatica was turned on, now get the useful data points. That is five seconds after the 
#submit times. For HR, since one sample per second, the sample number komt overeen met de tijd in seconden. 

In [230]:
def get_submit_array(data_start_time, submit_times_file, seconds_before_first_submit):
    submit_times = []
    f = open(submit_times_file, "r")
    lines = f.readlines()
    f.close()
    for each_time in lines[:3]:
#         each_time = each_time.strip()
        each_formatted_time = datetime.strptime(each_time[:-1], "%H:%M:%S")
        each_formatted_time = each_formatted_time.replace(year=(data_start_time.year),
                                                          month=(data_start_time.month),day=(data_start_time.day))
        submit_times.append(each_formatted_time)
    start_time =  submit_times[0]-(datetime.strptime(str(seconds_before_first_submit), "%S"))
    submit_times.insert(0, datetime.strptime(str(start_time)[-8:].strip(), "%H:%M:%S"))
    submit_times[0] = submit_times[0].replace(year=(data_start_time.year),
                                                          month=(data_start_time.month),day=(data_start_time.day))
    return submit_times[:3]

In [229]:
def get_empatica_startime_andHRdata(csvfile):
    all_data = read_csv_data(csvfile)
        
    StringTS = str(all_data[0])
    BetterStringTS = StringTS[2:12] #stomme haakjes weghalen die door append met csv komen +2 dus van wat logisch is

    ts = int(BetterStringTS)
    utc = datetime.utcfromtimestamp(ts)
    empaticaStart = utc + timedelta(hours=2)
    return empaticaStart, [item for sublist in all_data for item in sublist][2:]

In [52]:
def get_empatica_EDAdata(csvfile):
    all_data = read_csv_data(csvfile)
    all_data = [item for sublist in all_data for item in sublist]
    return all_data[2:]

In [233]:
def get_empatica_data(submit_times_file, HR_dataCSV_file, EDA_dataCSV_file, seconds_window):
    empatica_start, HR_data = get_empatica_startime_andHRdata(HR_dataCSV_file)
    EDA_data = get_empatica_EDAdata(EDA_dataCSV_file)
    submit_times = get_submit_array(empatica_start, submit_times_file, 30)
    
    HR_useful_datapoints = []
    EDA_useful_datapoints = []
    
    for each_sub_time in submit_times:
        time_diff = each_sub_time - empatica_start
        HRdata_idx = get_sec(time_diff) 
        EDAdata_idx = HRdata_idx*4
        curr_sub_time_HRdata = []
        curr_sub_time_EDAdata = []
        for i in range(seconds_window):
            curr_sub_time_HRdata.append(HR_data[HRdata_idx+i-1]) #-1 because indices start at zero and data_idx and time_diff is assumed to always be greater than 0.
        for i in range(seconds_window*4): # 4Hz is the sampling rate for EDA data            
            curr_sub_time_EDAdata.append(EDA_data[EDAdata_idx+i-1])
        HR_useful_datapoints.append(curr_sub_time_HRdata)
        EDA_useful_datapoints.append(curr_sub_time_EDAdata)
        
    return HR_useful_datapoints, EDA_useful_datapoints
    

In [8]:
HR_data, EDA_data = get_empatica_data("SubmitTimes.txt", "HR.csv", "EDA.csv", 4)

In [9]:
print(HR_data)

[['53.40', '53.50', '53.62', '53.77'], ['54.87', '54.83', '54.80', '54.77'], ['52.03', '52.15', '52.25', '52.33']]


In [10]:
print(EDA_data)

[['1.537292', '1.547538', '1.548818', '1.547538', '1.546257', '1.547538', '1.539853', '1.538573', '1.552660', '1.541134', '1.547538', '1.550099', '1.541134', '1.539853', '1.550099', '1.539853'], ['1.510397', '1.503993', '1.500151', '1.509116', '1.502713', '1.501432', '1.502713', '1.507835', '1.497590', '1.501432', '1.506555', '1.500151', '1.511678', '1.515520', '1.501432', '1.506555'], ['1.550099', '1.551380', '1.542415', '1.542415', '1.551380', '1.547538', '1.543695', '1.544976', '1.551380', '1.551380', '1.547538', '1.553941', '1.542415', '1.552660', '1.552660', '1.553941']]


In [11]:
import pandas as pd

In [13]:
pd.DataFrame(HR_data)

,0,1,2,3
0,53.40,53.50,53.62,53.77
1,54.87,54.83,54.80,54.77
2,52.03,52.15,52.25,52.33


In [167]:
import os
def get_immediate_subdirectories(root):
    return [name for name in os.listdir(root) if os.path.isdir(os.path.join(root, name))]

In [214]:
def get_empaticadata_for_all_participants(root_dir):
    sub_dirs = get_immediate_subdirectories(root_dir)
    all_data = []
    for each_dir in sub_dirs:
        participant_path = os.path.join(root_dir, each_dir)
        os.chdir(participant_path)
        print(os.getcwd())
        curr_HR_data, curr_EDA_data = get_empatica_data("SubmitTimes.txt", "HR.csv", "EDA.csv", 5)
        curr_participant_data = []
        curr_participant_data.append([curr_HR_data])
        curr_participant_data.append([curr_EDA_data])
        all_data.append(curr_participant_data)
    correctly_shaped_data = []
    
    #### INDEX KEY FOR all_data ####
    # first index = participant number 
    # second index = 0 for HR_data and 1 for EDA_data
    # third_index = always 0 for removing extra array (redundant array caused by code somewhere) -- can be ignored
    # fourth_index = which submission --> 0,1,2
    for i in range(len(sub_dirs)):
        for j in range(3):
#             correctly_shaped_data.append(all_data[i][0][0][j])
#             correctly_shaped_data.append(all_data[i][1][0][j])
            correctly_shaped_data.append(all_data[i][0][0][j]+all_data[i][1][0][j])
    return correctly_shaped_data
        


In [234]:
root_dir = "C:\\Users\\suhai\\Downloads\\EmpaticaDataFinal"
all_participants_data = get_empaticadata_for_all_participants(root_dir)

C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant03
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant04
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant07
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant08
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant09
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant10
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant11
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant12
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant13
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant14
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant15
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant16
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant17
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant18
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant19
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant20
C:\Users\suhai\Downloads\EmpaticaDataFinal\Participant21
C:\Users\suhai\Downloads\Empati

In [235]:
column_names = ["EDA_1", "EDA_2", "EDA_3", "EDA_4", "EDA_5", "EDA_6", "EDA_7",
                "EDA_8","EDA_9","EDA_10","EDA_11","EDA_12","EDA_13","EDA_14","EDA_15","EDA_16","EDA_17","EDA_18","EDA_19", "EDA_20"]

all_data_dataframe = pd.DataFrame(all_participants_data, columns=column_names)

In [209]:
all_data_dataframe

,HR_1,HR_2,HR_3,HR_4,HR_5,EDA_1,EDA_2,EDA_3,EDA_4,EDA_5,...,EDA_11,EDA_12,EDA_13,EDA_14,EDA_15,EDA_16,EDA_17,EDA_18,EDA_19,EDA_20
0,89.52,90.42,91.77,93.15,94.53,0.002563,0.001281,0.003844,0.000000,0.003844,...,0.002563,0.003844,0.001281,0.005125,0.000000,0.002563,0.001281,0.001281,0.002563,0.002563
1,142.48,143.62,144.23,145.43,146.05,0.005125,0.003844,0.003844,0.002563,0.001281,...,0.002563,0.003844,0.002563,0.003844,0.003844,0.001281,0.002563,0.002563,0.005125,0.000000
2,152.12,151.07,150.02,148.97,147.87,0.000000,0.002563,0.000000,0.002563,0.001281,...,0.002563,0.000000,0.003844,0.001281,0.003844,0.001281,0.002563,0.001281,0.003844,0.015376
3,57.23,56.98,57.03,56.93,56.83,0.002563,0.001281,0.003844,0.000000,0.005125,...,0.006406,0.003844,0.002563,0.005125,0.002563,0.005125,0.002563,0.003844,0.001281,0.002563
4,57.07,57.07,57.07,57.07,57.00,0.003844,0.001281,0.003844,0.002563,0.003844,...,0.000000,0.003844,0.001281,0.002563,0.001281,0.003844,0.000000,0.002563,0.002563,0.002563
5,54.28,54.33,54.38,54.43,54.53,0.000000,0.002563,0.005125,0.002563,0.003844,...,0.003844,0.000000,0.005125,0.006406,0.002563,0.003844,0.002563,0.003844,0.002563,0.003844
6,65.98,65.98,65.95,65.92,65.87,1.171049,1.168488,1.160803,1.160803,1.158241,...,1.194105,1.194105,1.192824,1.183858,1.185139,1.186420,1.177454,1.169769,1.164645,1.160803
7,68.67,68.85,69.02,69.20,69.37,1.265834,1.260710,1.253025,1.251744,1.250463,...,1.281204,1.279923,1.276081,1.273519,1.272238,1.273519,1.272238,1.282485,1.291451,1.295294
8,89.67,89.95,90.20,90.43,90.68,1.479838,1.468310,1.461906,1.455501,1.455501,...,1.468310,1.470872,1.483680,1.487523,1.483680,1.478557,1.478557,1.479838,1.478557,1.474714
9,74.87,74.98,75.10,75.20,75.30,0.087099,0.092223,0.090942,0.088380,0.087099,...,0.092223,0.088380,0.088380,0.092223,0.092223,0.087099,0.089661,0.092223,0.092223,0.088380


In [236]:
all_data_dataframe.to_csv('empaticaDataFinal.csv', index=False, header=True)